In [292]:
import requests 
import json
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from IPython.display import HTML, display
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [293]:
!pip install beautifulsoup4

In [294]:
!pip install lxml

In [295]:
!pip install html5lib


In [296]:
from bs4 import BeautifulSoup

#### Weblink of toronto postal codes

In [297]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#### Scrape the website and read the postal code table

In [313]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
results = soup.find('table', attrs={'class': 'wikitable sortable'})

rows = results.find_all('tr')
data = []

cols = rows[0].find_all('th')
headers = [ele.text.strip() for ele in cols]

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

result = pd.DataFrame(data, columns=headers)
result = result[result['Borough']!="Not assigned"].reset_index(drop=True)
result.dtypes

AttributeError: 'NoneType' object has no attribute 'find_all'

In [314]:
result.shape

(210, 5)

In [315]:

#geolocator = Nominatim(user_agent="Toronto_explorer")
#for code in result['Postcode']:
#    longlat = None
#    while (longlat is None):
#        print(code)
#        longlat = geolocator.geocode('{}, Toronto, ontario'.format(code))
#    lat.append(longlat.latitude)   
#    long.append(longlat.longitude)
#    print(lat)


#### Get the longitude and latitude from the geospatial file

In [316]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
longlat_dict = {}
for code, lt, ln in zip(geo_df['Postal Code'], geo_df['Latitude'], geo_df['Longitude']):
     longlat_dict[code] = [lt,ln]

#### Create columns longitude and latitude from geospatial data frame 

In [317]:
lat=[]
long=[]
for code in result['Postcode']:
    lonlat = longlat_dict[code]
    lat.append(lonlat[0])
    long.append(lonlat[1])

#### Add the location details to result data frame

In [318]:
result['Latitude'] = lat
result['Longitude'] = long
result

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
206,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999


#### Get the longitude and latitude values of Toronto, Ontario

In [320]:
address = 'Toronto, On'
geolocator =  Nominatim(user_agent="Toronto_Explorer")
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude
print(latitude, longitude)

43.653963 -79.387207


#### Create the toronto neighborhood map for analysis

In [321]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start=12)

for borough, neighbourhood, lat, lng in zip(result['Borough'], result['Neighbourhood'], result['Latitude'], result['Longitude']):
    label = '{}, {}'.format(borough, neighbourhood)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup = folium.Popup(label, parse_html=True), 
        color='blue',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,      
        parse_html=False).add_to(map_toronto) 
   
    #label = folium.Popup(label,max_width=450, parse_html=True)
map_toronto

#### Four suqare credentials 

In [322]:
CLIENT_ID = 'HQFIE2LGCMTESBTMPHN23QEPWMUYLRZDZ3AMZD4FWF5Z3QJA' # your Foursquare ID
CLIENT_SECRET = 'FAYQ2B50PC42VERLNA1MDG0HWDKMKRPC4ZAVJREEBDD13HEM' # your Foursquare Secret
VERSION = '20191204' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HQFIE2LGCMTESBTMPHN23QEPWMUYLRZDZ3AMZD4FWF5Z3QJA
CLIENT_SECRET:FAYQ2B50PC42VERLNA1MDG0HWDKMKRPC4ZAVJREEBDD13HEM


#### Exploring borough which we are interested

In [340]:
borough_to_explore = 'Downtown Toronto'
dtown_toronto = result[result['Borough'] == borough_to_explore].reset_index(drop=True)
dtown_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568
8,M5H,Downtown Toronto,King,43.650571,-79.384568
9,M5H,Downtown Toronto,Richmond,43.650571,-79.384568


In [330]:
address = borough_to_explore + ',Toronto' 

geolocator = Nominatim(user_agent = "dtown_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6563221 -79.3809161


#### Neighbourhood map of selected borough

In [341]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start=12)

for borough, neighbourhood, lat, lng in zip(dtown_toronto['Borough'], dtown_toronto['Neighbourhood'],
                                            dtown_toronto['Latitude'], dtown_toronto['Longitude']):
    label = '{}, {}'.format(borough, neighbourhood)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup = folium.Popup(label, parse_html=True), 
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,      
        parse_html=False).add_to(map_toronto) 
   
    #label = folium.Popup(label,max_width=450, parse_html=True)
map_toronto

#### Create URL for searching venues

In [342]:
radius = 500
limit = 100

url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude,VERSION, radius, limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=HQFIE2LGCMTESBTMPHN23QEPWMUYLRZDZ3AMZD4FWF5Z3QJA&client_secret=FAYQ2B50PC42VERLNA1MDG0HWDKMKRPC4ZAVJREEBDD13HEM&ll=43.6563221,-79.3809161&v=20191204&radius=500&limit=100'

#### Get the response from the URL 

In [343]:
response = requests.get(url).json()
response['response']['venues']
nearby_venues = json_normalize(response['response']['venues'])

#### Extract the categories from the venue

In [344]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [345]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Shark Club Sports Bar & Grill,Sports Bar,43.656381,-79.380921
1,Harvey's,Fast Food Restaurant,43.656341,-79.380947
2,Dundas Subway Station,Metro Station,43.656096,-79.380785
3,Spring Sushi,Sushi Restaurant,43.656253,-79.380660
4,Milestones,Fast Food Restaurant,43.656308,-79.380822
...,...,...,...,...
95,Tim Hortons,Coffee Shop,43.656044,-79.380793
96,Sack Goldblatt Mitchell LLP,None,43.656429,-79.381271
97,Pollination Media,Office,43.656455,-79.380652
98,ICE – International Currency Exchange - Bureau...,Currency Exchange,43.656261,-79.380473


#### Function to get the near by venues of the neighbourhoods using foursquare API

In [347]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        results
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [348]:
dtown_toronto_venues = getNearbyVenues(names=dtown_toronto['Neighbourhood'],
                                   latitudes=dtown_toronto['Latitude'],
                                   longitudes=dtown_toronto['Longitude']
                                  )

Harbourfront
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [349]:
dtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [213]:
dtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
CN Tower,14,14,14,14,14,14
Cabbagetown,42,42,42,42,42,42
Central Bay Street,83,83,83,83,83,83
Chinatown,93,93,93,93,93,93
Christie,16,16,16,16,16,16
Church and Wellesley,84,84,84,84,84,84


#### Add the venue categories as features using dummies

In [214]:
dtown_toronto_onehot = pd.get_dummies(dtown_toronto_venues['Venue Category'])
dtown_toronto_onehot['Neighbourhood'] = dtown_toronto_venues['Neighborhood']

In [215]:
fixed_columns = [dtown_toronto_onehot.columns[-1]] + list(dtown_toronto_onehot.columns[:-1])
dtown_toronto_onehot = dtown_toronto_onehot[fixed_columns]
dtown_toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group the neighbourhoods by mean for calculation

In [244]:
dtown_toronto_grouped = dtown_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
dtown_toronto_grouped.dtypes
#for ind in np.arange(dtown_toronto_grouped.shape[0]):
#     print(dtown_toronto_grouped.iloc[ind])
      

Neighbourhood             object
Afghan Restaurant        float64
Airport                  float64
Airport Food Court       float64
Airport Gate             float64
                          ...   
Video Game Store         float64
Vietnamese Restaurant    float64
Wine Bar                 float64
Wings Joint              float64
Yoga Studio              float64
Length: 207, dtype: object

In [245]:
#dtown_toronto_grouped = dtown_toronto_grouped.iloc[:,1:]
#dtown_toronto_grouped 

#### Get the top 'k' venues of the neighbourhood 

In [246]:
num_of_topvenues =10
for venue in dtown_toronto_grouped['Neighbourhood']:
    temp = dtown_toronto_grouped[dtown_toronto_grouped['Neighbourhood'] == venue].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
   # print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_of_topvenues))

In [351]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [352]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_of_topvenues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dtown_toronto_grouped['Neighbourhood']

for ind in np.arange( dtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtown_toronto_grouped.iloc[ind, :], num_of_topvenues)
    
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
2,Berczy Park,Coffee Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Café,Cheese Shop,Farmers Market,Cocktail Bar,Concert Hall
3,CN Tower,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
4,Cabbagetown,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Bakery,Pizza Place,Outdoor Sculpture,Butcher,Caribbean Restaurant


#### Cluster the neighbourhoods

In [353]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dtown_toronto_grouped_clustering = dtown_toronto_grouped.drop('Neighbourhood', 1)
dtown_toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 2, 1, 1, 3, 4, 1, 1], dtype=int32)

#### Insert cluster label into the result

In [354]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtown_toronto_merged = dtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtown_toronto_merged = dtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtown_toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Dessert Shop
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Bakery,Middle Eastern Restaurant,Café,Diner,Pizza Place,Bookstore
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Bakery,Middle Eastern Restaurant,Café,Diner,Pizza Place,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Diner,Clothing Store,Hotel,Park,Beer Bar
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Café,Cheese Shop,Farmers Market,Cocktail Bar,Concert Hall


In [355]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtown_toronto_merged['Latitude'], dtown_toronto_merged['Longitude'], dtown_toronto_merged['Neighbourhood'], dtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters


#### Cluster 1

In [356]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 0, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Downtown Toronto,Rosedale,0,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 2

In [357]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 1, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,1,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Dessert Shop
1,Downtown Toronto,Ryerson,1,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Bakery,Middle Eastern Restaurant,Café,Diner,Pizza Place,Bookstore
2,Downtown Toronto,Garden District,1,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Bakery,Middle Eastern Restaurant,Café,Diner,Pizza Place,Bookstore
3,Downtown Toronto,St. James Town,1,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Diner,Clothing Store,Hotel,Park,Beer Bar
4,Downtown Toronto,Berczy Park,1,Coffee Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Café,Cheese Shop,Farmers Market,Cocktail Bar,Concert Hall
5,Downtown Toronto,Central Bay Street,1,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Café,Japanese Restaurant,Restaurant,Spa,Bar
7,Downtown Toronto,Adelaide,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
8,Downtown Toronto,King,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
9,Downtown Toronto,Richmond,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
10,Downtown Toronto,Harbourfront East,1,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Italian Restaurant,Sporting Goods Shop


#### Cluster 3

In [358]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 2, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Downtown Toronto,CN Tower,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
23,Downtown Toronto,Bathurst Quay,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
24,Downtown Toronto,Island airport,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
25,Downtown Toronto,Harbourfront West,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
26,Downtown Toronto,King and Spadina,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
27,Downtown Toronto,Railway Lands,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate
28,Downtown Toronto,South Niagara,2,Airport Lounge,Airport Service,Boat or Ferry,Harbor / Marina,Boutique,Bar,Plane,Sculpture Garden,Airport,Airport Gate


#### Cluster 4

In [359]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 3, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,Harbord,3,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Italian Restaurant,Chinese Restaurant,Dessert Shop
18,Downtown Toronto,University of Toronto,3,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Italian Restaurant,Chinese Restaurant,Dessert Shop
19,Downtown Toronto,Chinatown,3,Café,Bar,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Chinese Restaurant,Coffee Shop,Bakery,Comfort Food Restaurant,Park
20,Downtown Toronto,Grange Park,3,Café,Bar,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Chinese Restaurant,Coffee Shop,Bakery,Comfort Food Restaurant,Park
21,Downtown Toronto,Kensington Market,3,Café,Bar,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Chinese Restaurant,Coffee Shop,Bakery,Comfort Food Restaurant,Park


#### Cluster 5

In [360]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 4, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,Christie,4,Café,Grocery Store,Park,Nightclub,Candy Store,Restaurant,Diner,Baby Store,Italian Restaurant,Coffee Shop
